# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [27]:
training.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|      1|     5|874965758|
|      1|      2|     3|876893171|
|      1|      3|     4|878542960|
|      1|      4|     3|876893119|
|      1|      5|     3|889751712|
+-------+-------+------+---------+
only showing top 5 rows



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



In [28]:
items.show(5)

+-------+-----------------+
|item_id|            movie|
+-------+-----------------+
|      1| Toy Story (1995)|
|      2| GoldenEye (1995)|
|      3|Four Rooms (1995)|
|      4|Get Shorty (1995)|
|      5|   Copycat (1995)|
+-------+-----------------+
only showing top 5 rows



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [10]:
training.count()

80000

In [12]:
# YOUR CODE HERE

training_size = training.count()
test_size = test.count()
items_size = items.count()

print(f'Number of ratings in training set: {training_size}')
print(f'Number of ratings in test set: {test_size}')
print(f'Number of movies: {items_size}')

Number of ratings in training set: 80000
Number of ratings in test set: 20000
Number of movies: 1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [13]:
type(training)

pyspark.sql.dataframe.DataFrame

In [17]:
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [18]:
# YOUR CODE HERE
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1162022967784533


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [34]:
predictions.agg(max('prediction')).show()

+---------------+
|max(prediction)|
+---------------+
|      12.008887|
+---------------+



In [40]:
training.filter(training['user_id'] == 471).show(50)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    471|      1|     4|889827881|
|    471|      8|     5|889827881|
|    471|     50|     3|889827757|
|    471|     71|     3|889828154|
|    471|     82|     5|889827822|
|    471|     94|     5|889828081|
|    471|     95|     4|889827822|
|    471|     99|     2|889827918|
|    471|    102|     5|889828081|
|    471|    140|     5|889827918|
|    471|    151|     2|889828154|
|    471|    172|     4|889827822|
|    471|    225|     5|889828026|
|    471|    393|     5|889827918|
|    471|    404|     2|889827757|
|    471|    418|     3|889827757|
|    471|    420|     1|889828027|
|    471|    422|     5|889827982|
|    471|    432|     1|889827822|
|    471|    465|     5|889827822|
|    471|    477|     5|889827918|
|    471|    501|     3|889828027|
|    471|    588|     1|889827881|
|    471|    596|     1|889827881|
|    471|    627|     1|889827881|
|    471|    768|   

In [26]:
# YOUR CODE HERE
model.recommendForAllUsers(10).show(10, False)

+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|recommendations                                                                                                                                                                       |
+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|471    |[{351, 12.190597}, {791, 11.702822}, {1036, 11.221279}, {1131, 10.572609}, {1475, 10.351559}, {916, 9.749181}, {1296, 9.662729}, {1311, 9.319625}, {1032, 8.981149}, {557, 8.964233}] |
|463    |[{854, 6.2063646}, {904, 6.179234}, {767, 5.947802}, {1187, 5.9210277}, {884, 5.5009236}, {987, 5.475616}, {292, 5.430072}, {1394, 5.4013596}, {1252, 5.360349}, {1107, 5.3483872}]   |
|833    |[{1368, 5.6462345}, {1205,

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!